In [2]:
# from the table of barcode and sample IDs
sample_ids_and_barcodes = """barcode 49 negative control	barcode 57 78020	barcode 65 78678	barcode 73 75028	barcode 81 77233	barcode 89 77718
barcode 50 76723	barcode 58 78022	barcode 66 78749	barcode 74 76831	barcode 82  77241	barcode 90 77724
barcode 51 76741	barcode 59 78025	barcode 67 78915	barcode 75 76904	barcode 83 77243	barcode 91 77731
barcode 52 76789	barcode 60 78026	barcode 68 78962	barcode 76 76926	barcode 84 77245	barcode 92 77751
barcode 53 76813	barcode 61 78085	barcode 69 78988	barcode 77 76986	barcode 85 77348	barcode 93 77046
barcode 54 77912	barcode 62 78184	barcode 70 78994	barcode 78 77019	barcode 86 77386	barcode 94 75079
barcode 55 78004	barcode 63 78190	barcode 71 79040	barcode 79 77158	barcode 87 77504	barcode 95 75094
barcode 56 78014	barcode 64 78262	barcode 72 79668	barcode 80 77185	barcode 88 77695	barcode 96 75096"""


In [3]:
pairs_list = []
for row in sample_ids_and_barcodes.split('\n'):
    elements = row.split('\t')
    pairs_list.extend(elements)
print(pairs_list)
mapping_rows = []
for entry in pairs_list:
    if 'negative control' in entry:
        entry = entry.replace('negative control', 'negative_control')
    fields = entry.split()
    barcode_name = fields[0] + fields[1]
    sample_id = fields[2]
    mapping_rows.append([barcode_name, sample_id])

with open('data/mapping.tsv', 'w') as output_file:
    for row in mapping_rows:
        print('\t'.join(row), file=output_file)

['barcode 49 negative control', 'barcode 57 78020', 'barcode 65 78678', 'barcode 73 75028', 'barcode 81 77233', 'barcode 89 77718', 'barcode 50 76723', 'barcode 58 78022', 'barcode 66 78749', 'barcode 74 76831', 'barcode 82  77241', 'barcode 90 77724', 'barcode 51 76741', 'barcode 59 78025', 'barcode 67 78915', 'barcode 75 76904', 'barcode 83 77243', 'barcode 91 77731', 'barcode 52 76789', 'barcode 60 78026', 'barcode 68 78962', 'barcode 76 76926', 'barcode 84 77245', 'barcode 92 77751', 'barcode 53 76813', 'barcode 61 78085', 'barcode 69 78988', 'barcode 77 76986', 'barcode 85 77348', 'barcode 93 77046', 'barcode 54 77912', 'barcode 62 78184', 'barcode 70 78994', 'barcode 78 77019', 'barcode 86 77386', 'barcode 94 75079', 'barcode 55 78004', 'barcode 63 78190', 'barcode 71 79040', 'barcode 79 77158', 'barcode 87 77504', 'barcode 95 75094', 'barcode 56 78014', 'barcode 64 78262', 'barcode 72 79668', 'barcode 80 77185', 'barcode 88 77695', 'barcode 96 75096']


In [4]:
import pandas as pd
import numpy as np

In [5]:
# Ct values not used in final submission
ct_values_df = pd.read_excel('data/verification 2 data.xlsx', sheet_name='Ct values', index_col=0)
ct_values_df.index = ct_values_df.index.map(str)
ct_values_df.loc['75028','n'] = '21.82'
ct_values_df.loc['75028','orf'] = '23.458'

In [16]:
# NB
# turn the list of sample IDs and barcodes into a pandas dataframe
# this output is used to make a CSV file mapping file used to rename the sequences
barcode_mapping_df = pd.DataFrame(mapping_rows, columns=['seqName', 'sample_id']).set_index('sample_id').drop('negative_control')
barcode_mapping_df.to_csv('data/sample_id_to_barcode.csv')


In [7]:
samples_barcodes_ct_df = barcode_mapping_df.join(ct_values_df)
samples_barcodes_ct_df.dtypes

seqName    object
n          object
orf        object
dtype: object

In [8]:
nextclade_df = pd.read_csv('data/nextclade_report.tsv', delimiter='\t')
nextclade_df.columns
samples_barcodes_ct_df.columns

Index(['seqName', 'n', 'orf'], dtype='object')

In [9]:
sample_nextclade_df = samples_barcodes_ct_df.reset_index().set_index('seqName').join(nextclade_df.set_index('seqName')).reset_index().set_index('sample_id')
selected_samples_df = sample_nextclade_df[sample_nextclade_df['totalMissing'] < 5000]

In [10]:
print('\n'.join(list(selected_samples_df['seqName'])))

barcode57
barcode73
barcode58
barcode66
barcode74
barcode51
barcode59
barcode67
barcode75
barcode52
barcode60
barcode68
barcode53
barcode69
barcode77
barcode54
barcode62
barcode70
barcode55
barcode63
barcode71
barcode79
barcode56
barcode64
barcode72
barcode80


In [11]:
selected_samples_df.to_excel('samples_for_uploading.xlsx')

In [12]:
nextclade_df.columns

Index(['seqName', 'clade', 'qc.overallScore', 'qc.overallStatus',
       'totalSubstitutions', 'totalDeletions', 'totalInsertions',
       'totalFrameShifts', 'totalAminoacidSubstitutions',
       'totalAminoacidDeletions', 'totalAminoacidInsertions', 'totalMissing',
       'totalNonACGTNs', 'totalPcrPrimerChanges', 'substitutions', 'deletions',
       'insertions', 'privateNucMutations.reversionSubstitutions',
       'privateNucMutations.labeledSubstitutions',
       'privateNucMutations.unlabeledSubstitutions',
       'privateNucMutations.totalReversionSubstitutions',
       'privateNucMutations.totalLabeledSubstitutions',
       'privateNucMutations.totalUnlabeledSubstitutions',
       'privateNucMutations.totalPrivateSubstitutions', 'frameShifts',
       'aaSubstitutions', 'aaDeletions', 'aaInsertions', 'missing',
       'nonACGTNs', 'pcrPrimerChanges', 'alignmentScore', 'alignmentStart',
       'alignmentEnd', 'qc.missingData.missingDataThreshold',
       'qc.missingData.score', '

In [13]:
# NB
# convert the metadata from CIFs to metadata relevant to GISAID

def convert_gender(g):
    if g == 'm':
        return 'Male'
    elif g == 'f':
        return 'Female'
    else:
        return 'Unknown'
    
def convert_district(d):
    d = d.replace("'", "")
    if ' ' in d:
        fields = d.split()
        d = ' '.join([field.capitalize() for field in fields])
    else:
        d = d.capitalize()
    if d == 'Butha Bothe':
        d = 'Butha Buthe'
    return d

patient_metadata = pd.read_excel('data/spreadsheet_with_metadata.xlsx', sheet_name='Sheet3', 
                                 converters = {'Sample ID': lambda s: str(s),
                                               'Gender':  convert_gender,
                                               'District': convert_district})
patient_metadata.columns
patient_metadata_subset = patient_metadata[['Sample ID', 'specimen collection', 'Specimen Type', 'Purpose of Sampling', 'Age', 'Gender', 'District']]
patient_metadata_subset
now = pd.Timestamp('now')
patient_metadata_subset.insert(len(patient_metadata_subset.columns), 'age', (now - patient_metadata_subset['Age']).astype('<m8[Y]').replace(-1, np.nan))
patient_metadata_subset = patient_metadata_subset.drop('Age', axis=1)
cols = list(patient_metadata_subset.columns)
cols[0] = 'sample_id'
patient_metadata_subset.columns = cols

In [17]:
# NB
# barcodes_to_submit is just a file with a list of barcodes, one per line
barcodes_to_submit = pd.DataFrame(open('data/barcodes_to_submit.txt').read().split('\n'), columns=['seqName'])
barcodes_to_submit_with_sample_ids = barcodes_to_submit.set_index('seqName').join(barcode_mapping_df.reset_index().set_index('seqName')).dropna()
sample_submission_info = barcodes_to_submit_with_sample_ids.reset_index().set_index('sample_id').join(patient_metadata_subset.set_index('sample_id')).reset_index()


In [15]:
# NB
# this takes the GISAID template and the metadata and creates the final GISAID submission XLSX
# in future the 'submitter' and 'covv_authors' can also be set here
lab_name = 'National Reference Laboratory - Ministry of Health Maseru Lesotho'
address = 'Botšabelo, Near Baylor College of Medicine, Maseru, Lesotho'
gisaid_df = pd.read_excel('data/bulk_upload_template.xlsx', sheet_name='Submissions', nrows=2,
                         converters={'covv_orig_lab_addr': lambda _: address,
                                     'covv_subm_lab_addr': lambda _: address,
                                     'covv_orig_lab': lambda _: lab_name,
                                     'covv_subm_lab': lambda _: lab_name,
                                     'covv_assembly_method': lambda _: 'Exatype',
                                     'covv_seq_technology': lambda _: 'Oxford Nanopore MinION',
                                     'covv_host': lambda _: 'human',
                                     'covv_type': lambda _: 'betacoronavirus',
                                     'covv_passage': lambda _: 'original',
                                     'covv_comment': lambda _: '',
                                     'comment_type': lambda _: '',
                                     'fn': lambda _:'all_sequences.fasta'})
gisaid_submit_entry = gisaid_df.iloc[0]

gisaid_submit_entries = []
for row in sample_submission_info.iterrows():
    row_info = row[1]
    virus_name = 'hCoV-19/Lesotho/L' + row_info['sample_id']
    submit_entry = gisaid_submit_entry.copy()
    submit_entry['covv_virus_name'] = virus_name
    submit_entry['covv_location'] = row_info['District']
    submit_entry['covv_specimen'] = row_info['Specimen Type']
    submit_entry['covv_subm_sample_id'] = 'L' + row_info['sample_id']
    gisaid_submit_entries.append(submit_entry)
    
final_gisaid_df = pd.concat(gisaid_submit_entries, axis=1).transpose()
final_gisaid_df.to_excel('data/gisaid_bulk_submission.xlsx')


,submitter,fn,covv_virus_name,covv_type,covv_passage,covv_location,covv_host,covv_specimen,covv_seq_technology,covv_assembly_method,covv_orig_lab,covv_orig_lab_addr,covv_subm_lab,covv_subm_lab_addr,covv_subm_sample_id,covv_authors,covv_comment,comment_type
0,Submitter,all_sequences.fasta,hCoV-19/Lesotho/L76723,betacoronavirus,original,Maseru,human,oropharangeal swab,Oxford Nanopore MinION,Exatype,National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",L76723,Authors,NaN,NaN
0,Submitter,all_sequences.fasta,hCoV-19/Lesotho/L76741,betacoronavirus,original,Berea,human,oropharangeal swab,Oxford Nanopore MinION,Exatype,National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",L76741,Authors,NaN,NaN
0,Submitter,all_sequences.fasta,hCoV-19/Lesotho/L76789,betacoronavirus,original,Maseru,human,nasopharangeal swab,Oxford Nanopore MinION,Exatype,National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",L76789,Authors,NaN,NaN
0,Submitter,all_sequences.fasta,hCoV-19/Lesotho/L76813,betacoronavirus,original,Qachas Nek,human,nasopharangeal swab,Oxford Nanopore MinION,Exatype,National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",L76813,Authors,NaN,NaN
0,Submitter,all_sequences.fasta,hCoV-19/Lesotho/L77912,betacoronavirus,original,Maseru,human,nasopharangeal swab,Oxford Nanopore MinION,Exatype,National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",L77912,Authors,NaN,NaN
0,Submitter,all_sequences.fasta,hCoV-19/Lesotho/L78004,betacoronavirus,original,Mafeteng,human,nasopharangeal swab,Oxford Nanopore MinION,Exatype,National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",L78004,Authors,NaN,NaN
0,Submitter,all_sequences.fasta,hCoV-19/Lesotho/L78014,betacoronavirus,original,Mafeteng,human,nasopharangeal swab,Oxford Nanopore MinION,Exatype,National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",L78014,Authors,NaN,NaN
0,Submitter,all_sequences.fasta,hCoV-19/Lesotho/L78020,betacoronavirus,original,Mafeteng,human,nasopharangeal swab,Oxford Nanopore MinION,Exatype,National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",L78020,Authors,NaN,NaN
0,Submitter,all_sequences.fasta,hCoV-19/Lesotho/L78022,betacoronavirus,original,Mafeteng,human,nasopharangeal swab,Oxford Nanopore MinION,Exatype,National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",L78022,Authors,NaN,NaN
0,Submitter,all_sequences.fasta,hCoV-19/Lesotho/L78025,betacoronavirus,original,Mafeteng,human,nasopharangeal swab,Oxford Nanopore MinION,Exatype,National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",National Reference Laboratory - Ministry of He...,"Botšabelo, Near Baylor College of Medicine, Ma...",L78025,Authors,NaN,NaN
